In [1]:
# treinar um modelo usando como base o livro do auto da compadecida
with open('data/auto_da_compadecida.txt', 'r', encoding='utf-8') as f:
    text = f.read()


In [2]:
print("Tamanho do datatset(caracteres): ", len(text))

Tamanho do datatset(caracteres):  74462


In [3]:
#Printa as primeiros 1000 caracteres
print(text[:1000])

                                              O AUTO DA COMPADECIDA  
                                                                        De Ariano Suassuna  
 
JOÃO GRILO – Então Chico o homem vem mesmo? Estou tão desconfiado, tu é tão sem confiança!  
CHICÓ – eu sem confiança? Que isso João está me desconhecendo? Pois eu juro como ele vem. Quer benzer 
o cachorro da mulher pra ver se ele não morre. A dificuldade não é ele não vir, mas é o padre benzer o 
bicho.  
JOÃO GRILO – Não  vai benzer por que?Que é que tem de mais em se benzer um cachorro?  
CHICÓ – Bom, eu digo assim porque eu sei como esse povo é cheio de coisas. Eu mesmo já tive cavalo 
bento.  
JOÃO GRILO – Que é isso Chico? Já estou ficando por aqui com suas histórias e quando se pede uma 
explicação vem sempre com a mesma ladainha: “Não sei só sei que foi assim”  
CHICÓ – Mas se eu tive mesmo um cavalo meu filho, o que é que eu vou fazer? Vou mentir.  Dizer 
que não tive?  
JOÃO GRILO – Você vem sempre com uma histór

In [19]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hiperparametros
batch_size = 64 
block_size = 256 
max_iters = 1000
eval_interval = 100
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

In [5]:
# caracteres unicos do auto da compadecida
chars = sorted(list(set(text)))
vocab_size = len(chars)
# mapeamento dos caracteres em inteiros e vice-versa
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # codificador
decode = lambda l: ''.join([itos[i] for i in l]) # decodificador

In [6]:
# Dividir dataset em Treino e Validação
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.85*len(data)) # os primeiros 85% vão ser treino, e o resto validação
train_data = data[:n]
val_data = data[n:]

In [7]:
# Carregamento dos dados
def get_batch(split):
    # Gera um pequeno lote de dados com input x e alvo y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [8]:
#funcao para estimar a perda
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [9]:
class Head(nn.Module):
    """ Uma cabeça de autoatenção """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # entrada de tamanho (batch, etapa temporal, canais)
        # saída de tamanho (batch, etapa temporal, tamanho da cabeça)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # calcular os scores de atenção ("afinidades")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # realizar a agregação ponderada dos valores
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


In [10]:
class MultiHeadAttention(nn.Module):
    """ Múltiplas cabeças de autoatenção em paralelo """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [11]:
class FeedFoward(nn.Module):
    """ uma camada linear simples seguida de uma não linearidade """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


In [12]:
class Block(nn.Module):
    """ Bloco Transformer: comunicação seguida de computação """

    def __init__(self, n_embd, n_head):
        # n_embd: dimensão da incorporação, n_head: o número de cabeças desejadas
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [13]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # cada token lê diretamente os logits para o próximo token de uma tabela de busca
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # camada de normalização final
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # inicialização melhorada
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx e targets são ambos tensores (B,T) de inteiros
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx é um array (B, T) de índices no contexto atual
        for _ in range(max_new_tokens):
            # recorta idx para os últimos block_size tokens
            idx_cond = idx[:, -block_size:]
            # obtém as previsões
            logits, loss = self(idx_cond)
            # foca apenas no último passo de tempo
            logits = logits[:, -1, :] # se torna (B, C)
            # aplica softmax para obter probabilidades
            probs = F.softmax(logits, dim=-1) # (B, C)
            # amostra da distribuição
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # anexa o índice amostrado à sequência em execução
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel()
m = model.to(device)
# imprime o número de parâmetros no modelo
print(sum(p.numel() for p in m.parameters())/1e6, 'M parâmetros')


10.807385 M parâmetros


In [14]:
# Criar um PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [20]:
for iter in range(max_iters):

    # de vez em quando, avalie a perda nos conjuntos de treino e validação
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"Iteração {iter}: perda no treino {losses['train']:.4f}, perda na validação {losses['val']:.4f}")

    # amostra um lote de dados
    xb, yb = get_batch('train')

    # avalia a perda
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


Iteração 0: perda no treino 0.1327, perda na validação 2.5610
Iteração 100: perda no treino 0.0981, perda na validação 2.7461
Iteração 200: perda no treino 0.0845, perda na validação 2.8757
Iteração 300: perda no treino 0.0764, perda na validação 2.9906
Iteração 400: perda no treino 0.0729, perda na validação 3.0528
Iteração 500: perda no treino 0.0678, perda na validação 3.1451
Iteração 600: perda no treino 0.0653, perda na validação 3.2379
Iteração 700: perda no treino 0.0644, perda na validação 3.2673
Iteração 800: perda no treino 0.0609, perda na validação 3.3179
Iteração 900: perda no treino 0.0596, perda na validação 3.3329
Iteração 999: perda no treino 0.0589, perda na validação 3.3334


In [22]:
# generate from the model
prompt = 'João Grilo'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
print(decode(m.generate(context.unsqueeze(0), max_new_tokens=500)[0].tolist()))


João Grilo vamos e arelo,é lhe o padre promete pa rometer quanto  homem. Pobre de jeito 
senhor 
amar as pessous bim de fe passe tras no gente ambém vieram. Padre 
JOÃO GRILO – Ah é um grande joão Grilo me de repeito, Padre João!  
PADRE – Não mais nome não ser. Muito pra migo que eu padre nominha para a gente.  
JOÃO GRILO – Mue deiro de sconherro é se morrer. E o doutro não todo promarido? Foi e tu a me vinguraça!  
CHICÓ – Está aíbaços do tisso e padre essetá ai e queixai o bolvo?  
JOÃO GRILO – O cach


In [ ]:
open('exemplo.txt', 'w').write(decode(m.generate(context.unsqueeze(0), max_new_tokens=10000)[0].tolist()))